# BigQuery Natural Language Analysis with Gemini

このノートブックでは、Gemini APIを使用してBigQueryのSalesforceデータを自然言語で分析します。

## 主な機能
- 🗣️ **自然言語でのクエリ**: 日本語でデータに関する質問をするだけで分析可能
- 🤖 **自動SQL生成**: Gemini APIがスキーマを理解してSQLを生成
- 📊 **結果の可視化**: Plotlyによる自動グラフ生成
- 📝 **サマリー生成**: AIによる分析結果の要約
- 📈 **ダッシュボード**: 複数分析の統合表示
- 💾 **履歴管理**: 分析履歴の保存と参照

## 使用モデル（2025年1月最新）
- **Gemini 2.5 Pro** (gemini-2.5-pro-exp-0117): データ分析最適化モデル
- **Gemini 2.5 Flash** (gemini-2.5-flash-exp-0117): 高速バランス版
- **Gemini 2.0 Flash** (gemini-2.0-flash-exp-01-21): 超高速処理版

## 1. 環境設定とライブラリのインストール

In [ ]:
# 必要なライブラリをインストール
!pip install -q google-cloud-bigquery google-cloud-bigquery-storage pandas db-dtypes
!pip install -q google-generativeai
!pip install -q plotly matplotlib seaborn
!pip install -q pyarrow

## 2. Google Cloud認証とAPI設定

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# Google Cloud認証（Colab環境の場合）
try:
    from google.colab import auth
    auth.authenticate_user()
    print('✅ Google Cloud認証完了')
except ImportError:
    print('ℹ️ ローカル環境で実行中 - gcloud auth を使用')

# BigQueryクライアントの初期化
from google.cloud import bigquery
from google.cloud.exceptions import GoogleCloudError

PROJECT_ID = 'esperanto-drawer-prod'
DATASET_ID = 'dm_business_planning'

try:
    client = bigquery.Client(project=PROJECT_ID)
    print(f'✅ BigQueryクライアント初期化完了: {PROJECT_ID}')
except Exception as e:
    print(f'❌ BigQueryクライアント初期化エラー: {e}')
    client = None

## 3. Gemini API設定

In [ ]:
import google.generativeai as genai

# Gemini API キーの設定
def setup_gemini_api():
    """Gemini APIキーを設定し、モデルを初期化"""
    api_key = None
    
    # 1. Colabシークレットから取得を試みる
    try:
        from google.colab import userdata
        api_key = userdata.get('GEMINI_API_KEY')
        print('✅ Colabシークレットから API Key を取得しました')
    except:
        pass
    
    # 2. 環境変数から取得を試みる
    if not api_key:
        api_key = os.getenv('GEMINI_API_KEY')
        if api_key:
            print('✅ 環境変数から API Key を取得しました')
    
    # 3. 手動入力
    if not api_key:
        from getpass import getpass
        api_key = getpass('Gemini API Key を入力してください: ')
    
    genai.configure(api_key=api_key)
    
    # モデルの初期化（フォールバック付き）
    models_to_try = [
        ('gemini-2.5-pro-exp-0117', '最新データ分析特化版'),
        ('gemini-2.5-flash-exp-0117', '高速バランス版'),
        ('gemini-2.0-flash-exp-01-21', '超高速版'),
        ('gemini-1.5-pro', '安定版')
    ]
    
    for model_name, description in models_to_try:
        try:
            model = genai.GenerativeModel(model_name)
            print(f'✅ 使用モデル: {model_name} ({description})')
            return model
        except Exception as e:
            print(f'⚠️ {model_name} は利用できません: {e}')
            continue
    
    raise Exception("利用可能なGeminiモデルが見つかりません")

# Geminiモデルの初期化
model = setup_gemini_api()

## 4. テーブルスキーマの取得（改良版）

In [ ]:
def get_table_schema_safe(dataset_id: str, table_id: str) -> pd.DataFrame:
    """テーブルのスキーマ情報を安全に取得"""
    
    # 最も基本的なクエリから始める
    queries = [
        # Option 1: COLUMN_FIELD_PATHSを使用（推奨）
        f"""
        SELECT 
            column_name,
            data_type,
            field_path
        FROM `{PROJECT_ID}.{dataset_id}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
        WHERE table_name = '{table_id}'
        ORDER BY column_name
        """,
        # Option 2: COLUMNSを使用（フォールバック）
        f"""
        SELECT 
            column_name,
            data_type
        FROM `{PROJECT_ID}.{dataset_id}.INFORMATION_SCHEMA.COLUMNS`
        WHERE table_name = '{table_id}'
        ORDER BY column_name
        """,
        # Option 3: テーブルから直接スキーマを取得（最終手段）
        f"""
        SELECT *
        FROM `{PROJECT_ID}.{dataset_id}.{table_id}`
        LIMIT 0
        """
    ]
    
    for i, query in enumerate(queries, 1):
        try:
            print(f"試行 {i}/3: スキーマ取得中...")
            if i == 3:
                # Option 3の場合は、クエリ結果からスキーマを推定
                job = client.query(query)
                schema = []
                for field in job.schema:
                    schema.append({
                        'column_name': field.name,
                        'data_type': field.field_type,
                        'description': field.description or ''
                    })
                df = pd.DataFrame(schema)
            else:
                df = client.query(query).to_dataframe()
            
            # descriptionカラムがない場合は追加
            if 'description' not in df.columns:
                df['description'] = ''
            
            print(f"✅ スキーマ取得成功: {len(df)} カラム")
            return df
            
        except Exception as e:
            print(f"⚠️ 試行 {i} 失敗: {str(e)[:100]}")
            continue
    
    # すべて失敗した場合
    print("❌ スキーマ取得に失敗しました")
    return pd.DataFrame(columns=['column_name', 'data_type', 'description'])

# スキーマ情報を取得
print("=" * 50)
print("📊 Salesforce Account Mart スキーマ取得")
account_schema = get_table_schema_safe(DATASET_ID, 'salesforce_account_mart')

print("\n" + "=" * 50)
print("📊 Salesforce Opportunity Mart スキーマ取得")
opportunity_schema = get_table_schema_safe(DATASET_ID, 'salesforce_opportunity_mart')

# 結果表示
print("\n" + "=" * 50)
print(f"✅ Account Mart: {len(account_schema)} カラム")
print(f"✅ Opportunity Mart: {len(opportunity_schema)} カラム")

# 主要カラムを表示
if not account_schema.empty:
    print("\n📋 Account Mart 主要カラム:")
    display(account_schema.head(10))

if not opportunity_schema.empty:
    print("\n📋 Opportunity Mart 主要カラム:")
    display(opportunity_schema.head(10))

## 5. 自然言語→SQL変換クラス（コア機能）

In [ ]:
class NLToBigQueryAnalyzer:
    """自然言語クエリをBigQuery SQLに変換して実行するクラス"""
    
    def __init__(self, client: bigquery.Client, model, 
                 account_schema: pd.DataFrame, 
                 opportunity_schema: pd.DataFrame):
        self.client = client
        self.model = model
        self.account_schema = account_schema
        self.opportunity_schema = opportunity_schema
        self.history = []  # 分析履歴
        
    def generate_sql(self, natural_language_query: str) -> str:
        """自然言語クエリからSQLを生成"""
        
        # スキーマ情報を文字列化（最大50カラムずつ）
        account_cols = self.account_schema[['column_name', 'data_type']].head(50).to_string(index=False)
        opportunity_cols = self.opportunity_schema[['column_name', 'data_type']].head(50).to_string(index=False)
        
        prompt = f"""
        あなたはBigQueryのSQLエキスパートです。
        以下のテーブルスキーマを使用して、ユーザーの質問に答えるSQLクエリを生成してください。
        
        ## 利用可能なテーブル:
        
        ### 1. `esperanto-drawer-prod.dm_business_planning.salesforce_account_mart`
        企業・顧客マスタデータ
        主要カラム:
        {account_cols}
        
        ### 2. `esperanto-drawer-prod.dm_business_planning.salesforce_opportunity_mart`
        商談・案件データ
        主要カラム:
        {opportunity_cols}
        
        ## 重要な注意事項:
        - 日付カラムは TIMESTAMP または DATE 型
        - 金額カラムは Amount, AnnualRevenue など
        - ステージ名は StageName
        - 必ず LIMIT 1000 を付ける（大量データ防止）
        - JOINする場合は AccountId で結合
        - エラーが起きにくいシンプルなクエリを心がける
        
        ## ユーザーの質問:
        {natural_language_query}
        
        SQLクエリのみを返してください。説明は不要です。
        SQLは```sql と ``` で囲んでください。
        """
        
        try:
            response = self.model.generate_content(prompt)
            sql = response.text
            
            # SQLを抽出
            if '```sql' in sql:
                sql = sql.split('```sql')[1].split('```')[0].strip()
            elif '```' in sql:
                sql = sql.split('```')[1].split('```')[0].strip()
                
            return sql
        except Exception as e:
            print(f"❌ SQL生成エラー: {e}")
            return None
    
    def execute_query(self, sql: str) -> Tuple[pd.DataFrame, str]:
        """SQLを実行してDataFrameを返す"""
        try:
            # ドライランでコスト確認
            job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
            dry_run_job = self.client.query(sql, job_config=job_config)
            
            bytes_processed = dry_run_job.total_bytes_processed
            gb_processed = bytes_processed / 1e9
            cost_estimate = gb_processed * 5  # $5 per TB
            
            print(f"💰 処理予定: {gb_processed:.3f} GB (推定コスト: ${cost_estimate:.4f})")
            
            # 実際に実行
            query_job = self.client.query(sql)
            df = query_job.to_dataframe()
            
            return df, None
            
        except Exception as e:
            error_msg = str(e)
            print(f"❌ クエリ実行エラー: {error_msg[:200]}")
            return pd.DataFrame(), error_msg
    
    def generate_summary(self, query: str, df: pd.DataFrame) -> str:
        """結果のサマリーを生成"""
        if df.empty:
            return "データが取得できませんでした。"
        
        # DataFrameの統計情報
        stats = {
            '行数': len(df),
            'カラム数': len(df.columns),
            'カラム名': list(df.columns)[:10]  # 最初の10カラム
        }
        
        # 数値カラムの基本統計
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            stats['数値カラム'] = list(numeric_cols)[:5]
        
        # サンプルデータ（最大10行）
        sample_data = df.head(10).to_string() if len(df) > 0 else "データなし"
        
        prompt = f"""
        以下のデータ分析結果を日本語で要約してください。
        
        元の質問: {query}
        
        データ統計:
        {json.dumps(stats, ensure_ascii=False, indent=2)}
        
        サンプルデータ（最大10行）:
        {sample_data[:2000]}
        
        以下の観点で3-5文で要約してください：
        1. 主要な発見・インサイト
        2. 数値的な傾向
        3. ビジネス上の示唆
        """
        
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"サマリー生成エラー: {e}"
    
    def analyze(self, natural_language_query: str, show_details: bool = True) -> Dict:
        """自然言語クエリを分析して結果を返す"""
        
        result = {
            'timestamp': datetime.now(),
            'query': natural_language_query,
            'sql': None,
            'data': pd.DataFrame(),
            'error': None,
            'summary': None
        }
        
        if show_details:
            print(f"\n{'='*60}")
            print(f"🔍 質問: {natural_language_query}")
            print("='*60}")
        
        # SQL生成
        print("\n⏳ SQL生成中...")
        sql = self.generate_sql(natural_language_query)
        if not sql:
            result['error'] = "SQL生成に失敗しました"
            return result
        
        result['sql'] = sql
        if show_details:
            print(f"\n📝 生成されたSQL:\n{sql}")
        
        # SQL実行
        print("\n⏳ クエリ実行中...")
        df, error = self.execute_query(sql)
        
        if error:
            result['error'] = error
            result['summary'] = f"エラーが発生しました: {error[:100]}"
        else:
            result['data'] = df
            print(f"✅ {len(df)} 行のデータを取得")
            
            # サマリー生成
            print("\n⏳ サマリー生成中...")
            result['summary'] = self.generate_summary(natural_language_query, df)
        
        # 履歴に追加
        self.history.append(result)
        
        return result

# アナライザーのインスタンス作成
if client and model:
    analyzer = NLToBigQueryAnalyzer(client, model, account_schema, opportunity_schema)
    print("\n✅ アナライザー初期化完了")
else:
    print("\n❌ アナライザー初期化失敗（BigQueryまたはGemini APIの設定を確認してください）")
    analyzer = None

## 6. 分析実行と可視化関数

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def run_analysis(question: str, show_chart: bool = True, chart_type: str = 'auto') -> Dict:
    """自然言語で分析を実行し、結果を可視化"""
    
    if not analyzer:
        print("❌ アナライザーが初期化されていません")
        return None
    
    # 分析実行
    result = analyzer.analyze(question)
    
    # 結果表示
    if result['summary']:
        print(f"\n📊 分析結果サマリー:")
        print(result['summary'])
    
    # データ表示
    if not result['data'].empty:
        print(f"\n📋 データ（上位10行）:")
        display(result['data'].head(10))
        
        # グラフ表示
        if show_chart:
            create_visualization(result['data'], question, chart_type)
    
    return result

def create_visualization(df: pd.DataFrame, title: str, chart_type: str = 'auto'):
    """データフレームから適切なグラフを自動生成"""
    
    if df.empty:
        return
    
    # 数値カラムと非数値カラムを識別
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
    
    # chart_typeが'auto'の場合、データから最適なグラフタイプを推定
    if chart_type == 'auto':
        if len(df) <= 20 and len(numeric_cols) > 0:
            chart_type = 'bar'
        elif len(numeric_cols) >= 2:
            chart_type = 'scatter'
        elif len(numeric_cols) == 1 and len(df) > 20:
            chart_type = 'line'
        else:
            chart_type = 'table'
    
    try:
        if chart_type == 'bar' and len(numeric_cols) > 0:
            # 棒グラフ
            x_col = non_numeric_cols[0] if non_numeric_cols else df.index
            y_col = numeric_cols[0]
            fig = px.bar(df.head(20), x=x_col, y=y_col, title=title)
            
        elif chart_type == 'line' and len(numeric_cols) > 0:
            # 線グラフ
            x_col = non_numeric_cols[0] if non_numeric_cols else df.index
            y_col = numeric_cols[0]
            fig = px.line(df, x=x_col, y=y_col, title=title, markers=True)
            
        elif chart_type == 'scatter' and len(numeric_cols) >= 2:
            # 散布図
            fig = px.scatter(df, x=numeric_cols[0], y=numeric_cols[1], title=title)
            
        elif chart_type == 'pie' and len(numeric_cols) > 0 and len(non_numeric_cols) > 0:
            # 円グラフ
            fig = px.pie(df.head(10), values=numeric_cols[0], names=non_numeric_cols[0], title=title)
            
        else:
            # テーブル表示
            fig = go.Figure(data=[go.Table(
                header=dict(values=list(df.columns),
                           fill_color='paleturquoise',
                           align='left'),
                cells=dict(values=[df[col] for col in df.columns],
                          fill_color='lavender',
                          align='left'))
            ])
            fig.update_layout(title=title)
        
        fig.show()
        
    except Exception as e:
        print(f"⚠️ グラフ生成エラー: {e}")

print("✅ 分析・可視化関数の準備完了")

## 7. ダッシュボード生成機能

In [ ]:
def create_dashboard(analyses: Dict[str, str] = None):
    """複数の分析結果を組み合わせたダッシュボードを作成"""
    
    if not analyzer:
        print("❌ アナライザーが初期化されていません")
        return None
    
    # デフォルトの分析セット
    if analyses is None:
        analyses = {
            "月別売上トレンド": "2024年の月別受注金額を集計",
            "ステージ別商談": "現在の商談をステージ別に件数と金額で集計",
            "顧客セグメント": "ENT、MID、SMBフラグ別の顧客数",
            "業界別分析": "業界別の顧客数上位10件"
        }
    
    results = {}
    print("📊 ダッシュボード生成中...\n")
    
    # 各分析を実行
    for title, query in analyses.items():
        print(f"⏳ 分析中: {title}")
        try:
            result = analyzer.analyze(query, show_details=False)
            results[title] = result
            if not result['data'].empty:
                print(f"✅ {title}: {len(result['data'])} 行取得")
            else:
                print(f"⚠️ {title}: データなし")
        except Exception as e:
            print(f"❌ {title}: エラー - {e}")
            results[title] = None
    
    # ダッシュボード作成
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=list(analyses.keys()),
        specs=[
            [{'type': 'bar'}, {'type': 'bar'}],
            [{'type': 'bar'}, {'type': 'scatter'}]
        ],
        vertical_spacing=0.15,
        horizontal_spacing=0.15
    )
    
    # 各グラフを配置
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    
    for (title, result), (row, col), color in zip(results.items(), positions, colors):
        if result and not result['data'].empty:
            df = result['data'].head(20)
            
            # カラムを特定
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
            
            if len(numeric_cols) > 0:
                y_col = numeric_cols[0]
                x_col = non_numeric_cols[0] if len(non_numeric_cols) > 0 else df.index
                
                if row == 2 and col == 2:  # 最後は散布図/線グラフ
                    fig.add_trace(
                        go.Scatter(
                            x=df[x_col] if isinstance(x_col, str) else x_col,
                            y=df[y_col],
                            mode='lines+markers',
                            name=title,
                            marker=dict(color=color)
                        ),
                        row=row, col=col
                    )
                else:  # 棒グラフ
                    fig.add_trace(
                        go.Bar(
                            x=df[x_col] if isinstance(x_col, str) else x_col,
                            y=df[y_col],
                            name=title,
                            marker=dict(color=color)
                        ),
                        row=row, col=col
                    )
    
    # レイアウト設定
    fig.update_layout(
        height=800,
        showlegend=False,
        title=dict(
            text="<b>Salesforce データ分析ダッシュボード</b>",
            font=dict(size=20),
            x=0.5,
            xanchor='center'
        ),
        hovermode='x unified'
    )
    
    # 軸ラベルを回転
    fig.update_xaxes(tickangle=45)
    
    fig.show()
    
    return results

print("✅ ダッシュボード機能の準備完了")

## 8. 履歴管理機能

In [ ]:
import pickle
from typing import List

class AnalysisHistory:
    """分析履歴の管理クラス"""
    
    def __init__(self, analyzer: NLToBigQueryAnalyzer = None):
        self.analyzer = analyzer
        self.history = analyzer.history if analyzer else []
    
    def show_history(self, last_n: int = 10):
        """最近の分析履歴を表示"""
        if not self.history:
            print("📭 履歴がありません")
            return
        
        print(f"\n📚 分析履歴（最新{last_n}件）")
        print("=" * 80)
        
        for i, item in enumerate(self.history[-last_n:], 1):
            print(f"\n[{i}] {item['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"   質問: {item['query']}")
            if not item['data'].empty:
                print(f"   結果: {len(item['data'])} 行 × {len(item['data'].columns)} カラム")
            else:
                print(f"   結果: データなし")
            if item['error']:
                print(f"   エラー: {item['error'][:100]}")
    
    def get_history_dataframe(self) -> pd.DataFrame:
        """履歴をDataFrameとして取得"""
        if not self.history:
            return pd.DataFrame()
        
        history_data = []
        for item in self.history:
            history_data.append({
                'timestamp': item['timestamp'],
                'query': item['query'],
                'rows': len(item['data']) if not item['data'].empty else 0,
                'columns': len(item['data'].columns) if not item['data'].empty else 0,
                'has_error': item['error'] is not None
            })
        
        return pd.DataFrame(history_data)
    
    def save(self, filename: str = 'analysis_history.pkl'):
        """履歴をファイルに保存"""
        try:
            with open(filename, 'wb') as f:
                pickle.dump(self.history, f)
            print(f"💾 履歴を {filename} に保存しました")
        except Exception as e:
            print(f"❌ 保存エラー: {e}")
    
    def load(self, filename: str = 'analysis_history.pkl'):
        """履歴をファイルから読み込み"""
        try:
            with open(filename, 'rb') as f:
                self.history = pickle.load(f)
            print(f"📂 履歴を {filename} から読み込みました（{len(self.history)}件）")
            if self.analyzer:
                self.analyzer.history = self.history
        except FileNotFoundError:
            print(f"⚠️ ファイル {filename} が見つかりません")
        except Exception as e:
            print(f"❌ 読み込みエラー: {e}")
    
    def export_to_csv(self, filename: str = 'analysis_history.csv'):
        """履歴をCSVファイルにエクスポート"""
        df = self.get_history_dataframe()
        if not df.empty:
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"📊 履歴を {filename} にエクスポートしました")
        else:
            print("📭 エクスポートする履歴がありません")

# 履歴管理インスタンスの作成
if analyzer:
    history_manager = AnalysisHistory(analyzer)
    print("✅ 履歴管理機能の準備完了")
else:
    history_manager = None
    print("⚠️ 履歴管理機能は利用できません（アナライザーが未初期化）")

## 9. 分析例の実行

In [ ]:
# 分析例1: 売上上位企業
result1 = run_analysis("年間売上が最も高い上位10社を教えて")

In [ ]:
# 分析例2: 月別トレンド
result2 = run_analysis("2024年の月別商談数の推移を見せて", chart_type='line')

In [ ]:
# 分析例3: セグメント分析
result3 = run_analysis("ENT、MID、SMBフラグ別の企業数と平均売上", chart_type='bar')

## 10. インタラクティブな分析

In [ ]:
# 自由に質問を入力して分析
question = input("分析したい内容を自然言語で入力してください: ")
if question:
    result = run_analysis(question)

## 11. ダッシュボード生成

In [ ]:
# カスタムダッシュボードの生成
dashboard_results = create_dashboard()

## 12. 履歴の管理と表示

In [ ]:
# 履歴を表示
if history_manager:
    history_manager.show_history(last_n=5)
    
    # 履歴をDataFrameとして取得
    history_df = history_manager.get_history_dataframe()
    if not history_df.empty:
        print("\n📊 履歴統計:")
        display(history_df.describe())

In [ ]:
# 履歴を保存
if history_manager:
    history_manager.save('analysis_history.pkl')
    history_manager.export_to_csv('analysis_history.csv')

## 13. 分析テンプレート集

In [ ]:
# よく使う分析のテンプレート
analysis_templates = {
    "営業パフォーマンス": [
        "今月のClosedWonの商談金額合計は？",
        "商談の平均クローズ期間をステージ別に集計",
        "今四半期の新規商談数と金額"
    ],
    "顧客分析": [
        "従業員数1000人以上の大企業リスト上位20社",
        "業界別の顧客数と平均売上",
        "地域別の顧客分布"
    ],
    "トレンド分析": [
        "過去12ヶ月の月別売上推移",
        "四半期別の新規顧客獲得数",
        "商談ステージ別の推移"
    ]
}

# テンプレート選択と実行
print("📋 利用可能なテンプレート:")
for i, category in enumerate(analysis_templates.keys(), 1):
    print(f"{i}. {category}")

try:
    choice = int(input("\nカテゴリ番号を選択 (1-3): "))
    if 1 <= choice <= 3:
        selected_category = list(analysis_templates.keys())[choice - 1]
        print(f"\n選択: {selected_category}")
        
        for question in analysis_templates[selected_category]:
            print(f"\n▶ {question}")
            try:
                result = run_analysis(question, show_chart=True)
            except Exception as e:
                print(f"❌ エラー: {e}")
except:
    print("スキップします")

## まとめ

このノートブックでは、以下の機能を実装しました：

✅ **実装済み機能**
1. 🗣️ **自然言語でのクエリ** - 日本語でデータに関する質問をするだけで分析可能
2. 🤖 **自動SQL生成** - Gemini APIがスキーマを理解してSQLを生成
3. 📊 **結果の可視化** - Plotlyによる自動グラフ生成
4. 📝 **サマリー生成** - AIによる分析結果の要約
5. 📈 **ダッシュボード** - 複数分析の統合表示
6. 💾 **履歴管理** - 分析履歴の保存と参照

### 使い方のヒント
- 具体的な数値や期間を含めると、より正確な分析ができます
- 「上位10件」「月別」「業界別」などの集計キーワードを使うと効果的です
- 複雑な分析は段階的に実行することをお勧めします

### トラブルシューティング
- **スキーマ取得エラー**: INFORMATION_SCHEMAの権限を確認
- **SQL実行エラー**: クエリの構文とテーブル名を確認
- **Gemini APIエラー**: API キーと利用制限を確認

### 注意事項
- 大量データのクエリはコストがかかるため、LIMIT句が自動追加されます
- Gemini APIの利用制限に注意してください
- 機密情報の取り扱いには十分注意してください